In [147]:
# HTTPS/Internet Data formats/structures/standards/constraints/encodings/decodings
import requests

In [148]:
# HTTP Requests:

# The generic request():
# requests.request(method: str, url: str, **kwags) -> requests.models.Response 

# Specific request methods:

# 'get' to a 'web-page'/address as 'r':
r = requests.get('http://api.github.com/events')
print('get:',r, ':', type(r))

# A web identifier/locator/address: 
# Alias: URL - Universal Resource Locator
addr = 'https://httpbin.org'

# 'post' data to an address:
r = requests.post(addr + '/post', data={'key':'value'})
print('post:',r, ':', type(r))

# 'put' data to an address:
r = requests.put(addr + '/put', data = {})
print('put:',r, ':', type(r))

# Q. What are the constraints on valid 'data' types?
    # {} is valid (dict)
    # 1 is invalid (int)

# 'delete' to an address:
r = requests.delete(addr+'/delete')
print('delete:',r, ':', type(r))

# 'head' to an address:
r = requests.head(addr + '/get')
print('head:',r, ':', type(r))

# 'options' to an address: ???
r = requests.options(addr + '/get')
print('options:',r, ':', type(r))

# 'patch' to an adress: requests.patch(url, data=None, **kwags) -> Response

# NOTE: As shown, Request methods return Response objects.

get: <Response [200]> : <class 'requests.models.Response'>
post: <Response [200]> : <class 'requests.models.Response'>
put: <Response [200]> : <class 'requests.models.Response'>
delete: <Response [200]> : <class 'requests.models.Response'>
head: <Response [200]> : <class 'requests.models.Response'>
options: <Response [200]> : <class 'requests.models.Response'>


In [149]:
# Hypothesis: 
    # Responses have a 'status_code' field/attribute ; (1)
    # status_code is an integer ; (2)
        # Status code 2?? <-> "Success"
        # Status code 3?? <-> "Redirection"
        # Status code 4?? <-> "Client Errors"
        # Status code 5?? <-> "Server Errors"

# Validating (1):
print("Status code:",r.status_code)

# Validating (2):
print("Type:",type(r.status_code))

Status code: 200
Type: <class 'int'>


In [150]:
# Parameterised URLs:
    # Parameters are encooded in the URL string

url = addr

# params are of Type dict((str:list(str)), ...) ? Is this the most generic form?
p = {'horse':'neigh', 'cat':'meow', 'dog':['bark','woof'], 'fox':None}
r = requests.get(url+'/get',params=p)
print("Parameterised URL:",r.url)

Parameterised URL: https://httpbin.org/get?horse=neigh&cat=meow&dog=bark&dog=woof


In [151]:
# Is data equivalent to params?
p = {'a':['b']}

# No. Do all distinctions derive from the url field of the Response?
# Does the distinction vary with the Request? eg. a 'get()' and  a 'post()'

print("GET Request:")

# data to GET:
r = requests.get(url+'/get', data=p, params=p)
print("URL:",r.url)
print(r.text)

print('\n\n')
print("POST Request:")

# params to POST:
r = requests.post(url+'/post', data=p, params=p)
print("PURL:",r.url)
print(r.text)

GET Request:
URL: https://httpbin.org/get?a=b
{
  "args": {
    "a": "b"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "3", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e84-1b1b7bac729981a2442923b5"
  }, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/get?a=b"
}




POST Request:
PURL: https://httpbin.org/post?a=b
{
  "args": {
    "a": "b"
  }, 
  "data": "", 
  "files": {}, 
  "form": {
    "a": "b"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "3", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e85-05429d1f5491c5057f3a8387"
  }, 
  "json": null, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/post?a=b"

In [152]:
# All Responses have a url field/attribute:
    # A url is a string.

url = addr
r = requests.put(url + '/put', data = {})

# Validating:
print(r.url)
print('Type:',type(r.url))

https://httpbin.org/put
Type: <class 'str'>


In [153]:
# In a URL, the params (ki,vi) are represented as strings of form:
    # base_URL + '?' + 'k1=v1' + '&' + 'k2=v2' + '&' + ...

# Q. What if the keys(ki) and values(vi) themselves contain 
# the strings '?', '&', '=' and so on?
p = {'?this=correct':'yes&no'}
r = requests.get(url+'/get',params=p)
print("Resultant PURL:",r.url)

Resultant PURL: https://httpbin.org/get?%3Fthis%3Dcorrect=yes%26no


In [154]:
# Hypothesis:
    # The strings are encoded as follows: (1)
        # '?' <-> '%3F'
        # '=' <-> '%3D'
        # '&' <-> '%26'

    # NOTE: Simple rule, but not necessarily true.
    # Each encoded character 'c' is of form : '%??'
    # where, ? belong to {0,1,...,9,A,B,...,F} ; (2)

# Q. If '%??' do not represent themselves, then what represents them?
p = {'%26and&':'?question%3F=equal%3D'}
r = requests.get(url+'/get',params=p)
print('Encoded URL:',r.url)

# Hypothesis:
    # The string '%' is encoded as '%25' ;
    # The string '%25' is encoded as '%2525' ;
    # The string '%2525' is encoded as '%252525' ; ... and so on

# Q. If '&' maps to '%26', then what maps to '&'? What about '%','=' and '?'?

# NOTE: Let S be the set of all strings. 
# An encoding(E) is defined as - E: S <-> S ;

# "To say less but mean more" is simply an encoding.
# To say more to mean less implies the ability to say less to mean more and vice-versa.

p = {'   ':'whitespace', '\t':'tab'}
r = requests.get(url+'/get',params=p)
print('Whitespace Encoding:',r.url)

p = {'+':'plus', '-':'minus'}
r = requests.get(url+'/get',params=p)
print('Plus/Minus Encoding:',r.url)

# Defining more encodings:
    # ' ' <-> '+'
    # '+' <-> '%2B'
    # '-' <-> '-'


Encoded URL: https://httpbin.org/get?%2526and%26=%3Fquestion%253F%3Dequal%253D
Whitespace Encoding: https://httpbin.org/get?+++=whitespace&%09=tab
Plus/Minus Encoding: https://httpbin.org/get?%2B=plus&-=minus


In [155]:
# Hypothesis:
p = {(url+'/get'):''}
print('params dict:',p)
try:
    r = requests.get('',params=p)
    print("Parameterised URLs are simply URLs:",r.url)
except Exception as e:
    print('[',e,"] error")
    print('Q. How to express any URL as a Parameterised URL?')

# My URL encoding is contradictory: '' -> '?' but not vice-versa! 

params dict: {'https://httpbin.org/get': ''}
[ Invalid URL '': No scheme supplied. Perhaps you meant https://? ] error
Q. How to express any URL as a Parameterised URL?


In [156]:
# Quote: (Let 'r' denote a Response)
"""
Requests will automatically decode content from the server. 
Most unicode charsets are seamlessly decoded.

When you make a request, Requests makes educated guesses 
about the encoding of the response based on the HTTP headers. 
The text encoding guessed by Requests is used when you access 
r.text. 

You can find out what encoding Requests is using, 
and change it, using the r.encoding property.
"""
# Takeaway: Responses have 'text' and 'encoding' fields.

# Hypothesis: 
    # text is a string ; (1)
    # encoding is a string ; (2)

# "Arbitrary" response:
url = "https://httpbin.org"
p = {'+':'plus', '-':'minus'}
r = requests.get(url+'/get',params=p)

# Validating (1)
print("text is :",type(r.text))
print(r.text)

# Validating (2)
print("encoding is:",type(r.encoding))
print(r.encoding)

# Although text is a string, it seems to not be ANY string.

text is : <class 'str'>
{
  "args": {
    "+": "plus", 
    "-": "minus"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e8d-64ab16140b61804e6456c8c5"
  }, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/get?%2B=plus&-=minus"
}

encoding is: <class 'str'>
utf-8


In [157]:
# Hypothesis:
    # Responses have 'content' field/attribute ; (1)
    # content is of variable/dynamic Type.

# Fixed testing URL:
url = "https://httpbin.org"

# "Arbitrary" Response 1:
r = requests.head(url + '/get')

# Validating:
print("Type:",type(r.content))
print(r.content)

print('======================================')

# "Arbitrary" Response 2:
r = requests.delete(url+'/delete')
print('Type:', type(r.content))
print(r.content)

# content is of Type : bytes

Type: <class 'bytes'>
b''
Type: <class 'bytes'>
b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "0", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.31.0", \n    "X-Amzn-Trace-Id": "Root=1-65dd7e8f-4a9c49ad3eac54651e05c26c"\n  }, \n  "json": null, \n  "origin": "14.97.164.46", \n  "url": "https://httpbin.org/delete"\n}\n'


In [158]:
# Hypothesis:
    # Response text and content are dependent on each other

url = "https://httpbin.org"

# "Arbitrary" Response:
r = requests.delete(url+'/delete')

# Content:
print('Content:')
print(r.content)

print('\n\n')

# Text:
print('Text:')
print(r.text)

Content:
b'{\n  "args": {}, \n  "data": "", \n  "files": {}, \n  "form": {}, \n  "headers": {\n    "Accept": "*/*", \n    "Accept-Encoding": "gzip, deflate", \n    "Content-Length": "0", \n    "Host": "httpbin.org", \n    "User-Agent": "python-requests/2.31.0", \n    "X-Amzn-Trace-Id": "Root=1-65dd7e90-7378a2914fba800e242bc1b4"\n  }, \n  "json": null, \n  "origin": "14.97.164.46", \n  "url": "https://httpbin.org/delete"\n}\n'



Text:
{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "0", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e90-7378a2914fba800e242bc1b4"
  }, 
  "json": null, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/delete"
}



In [159]:
# The "Content" of a Response(r):

# "Arbitrary" Response:
url = "https://httpbin.org"
r = requests.get(url+'/get',data={'a':2,'b':3}, params={'a':1})

# After transfer through HTTP, and decoding the 'gzip' and 'deflate' 
# encodings, we obtain the Binary Content as a sequence of bytes in r.content ;
# 'br' transfer encoding is also decoded in the process.

# Any valid "JSON Content" may be decoded further by using the json() method:
    # Response.json() is an instance method ;
    # It returns a dict Type object that represents the JSON;
# The JSON method raises an Exception in-case decoding fails ;

# Decoding the Response Content as JSON:
json_dat = None
try:
    json_dat = r.json()
    print(type(json_dat))
except Exception as e:
    print('Decoding Failed')
    print(e)

print(json_dat)

# Q. What is JSON? A rooted labelled tree ?

<class 'dict'>
{'args': {'a': '1'}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Content-Length': '7', 'Content-Type': 'application/x-www-form-urlencoded', 'Host': 'httpbin.org', 'User-Agent': 'python-requests/2.31.0', 'X-Amzn-Trace-Id': 'Root=1-65dd7e92-33e4de7537d457897fda63ff'}, 'origin': '14.97.164.46', 'url': 'https://httpbin.org/get?a=1'}


In [160]:
# The 'Content' of a Response:

# "Arbitrary" Response:
url = "https://httpbin.org"
r = requests.delete(addr+'/delete', stream = True)

# The 'stream' flag must be set to True of any Request, to obtain "Raw Content".
# The "Raw Content" of a Response is stored in the 'raw' attribute.
print(type(r.raw))

# The 'raw' content can be parsed ONCE sequentially, 
# in a "streamed" manner. Interestingly, invoking json() on 
# a Response, nullifies its 'raw' attribute.

# The 'Raw Content' may be parsed using different methods:
    # urllib3.response.HTTPResponse.read()
    # requests.models.Response.iter_content()



<class 'urllib3.response.HTTPResponse'>


In [161]:
# HTTP Requests are identified by 'headers'.
# Headers along with URLs define(completely?) a Request.
# (params, data, stream) also define a Request?

url = "https://httpbin.org"
hdr = {
    'user-agent': 'my-app/0.0.1', 
    'best-singer': 'Taylor Swift', 
    'company':'Sigmoid', 
    'Money':str(0), 
    # 'InvalidHeader Error, Must be of Type str or bytes'
    # 'Food':100, 
    # 'Time':{
    #     'work':50,
    #     'play':10,
    # },
}

r = requests.get(url+'/get',headers = hdr)
print(r.text)

# Quote:
"""
Note: Custom headers are given less precedence 
than more specific sources of information:

Example,

Authorization headers set with headers= will 
be overridden if credentials are specified in .netrc, 
which in turn will be overridden by the auth= parameter. 
Requests will search for the netrc file at ~/.netrc, ~/_netrc, or 
at the path specified by the NETRC environment variable.
"""

{
  "args": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Best-Singer": "Taylor Swift", 
    "Company": "Sigmoid", 
    "Host": "httpbin.org", 
    "Money": "0", 
    "User-Agent": "my-app/0.0.1", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e94-79a1dab805307b0a1ea468e0"
  }, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/get"
}



'\nNote: Custom headers are given less precedence \nthan more specific sources of information:\n\nExample,\n\nAuthorization headers set with headers= will \nbe overridden if credentials are specified in .netrc, \nwhich in turn will be overridden by the auth= parameter. \nRequests will search for the netrc file at ~/.netrc, ~/_netrc, or \nat the path specified by the NETRC environment variable.\n'

In [162]:
# Constraints on 'data' type:

url = 'https://httpbin.org'

# For post() requests, data may be a dict of any type?
dat = {None:[None,None], "0":{'':{'1':2,'2':1}, '2':[None,12], '\0':[None,[None,None]]}, 0:'1', '\0':[[1,2,3],2,3,4]}
r = requests.post(url+'/post', data=dat)
print(r.text)

# Observations:
    # 'data' seems to support ONLY data of type dict(str:list(str))
    # Some hierarchy is flattened and any types are casted to the above form.
    # Nested lists are dereferenced, but dictionaries are not dereferenced?

    # It occupies the 'form' field in the Response text/content.
    # (Quote: "Your dictionary of data will automatically be 
    # form-encoded when the request is made")

    # Key int(0) maps to Key str(int(0)) ; Does key a map to Key str(a) ?
    # None keys and None lists/values seem to be omitted entiely. 


{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "\u0000": [
      "1", 
      "2", 
      "3", 
      "2", 
      "3", 
      "4"
    ], 
    "0": [
      "", 
      "2", 
      "\u0000", 
      "1"
    ]
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "52", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e95-1d3fa3bb2e59ba110202e1a1"
  }, 
  "json": null, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/post"
}



In [163]:
# post() data may also be of type str:
r = requests.post(url+'/post', data="nothing")
print(r.text)

# Observations:
    # 'headers/Content-Type' missing
    # 'form' field nullified
    # 'data' field occupied

{
  "args": {}, 
  "data": "nothing", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "7", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e97-40db858c0d86d5f23be9a321"
  }, 
  "json": null, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/post"
}



In [164]:
# post() data may also be of the form: [(keyA,valA1), (keyA,valA2) ...]
# interpreted equivalently to the form: {keyA:[valA1, valA2] ...}
datA = [('a',2),('a',3),('b',None),('c',{'d':1,'e':2})]
rA = requests.post(url+'/post',data=datA)

datB = {'a':{2:None,3:None}, 'b':None, 'c':['d','e']}
rB = requests.post(url+'/post',data=datB)

# "X-Amzn-Trace-Id" fields are distinguishable ;
print("Are the two response text attributes equal:",rA.text==rB.text)

Are the two response text attributes equal: False


In [165]:
# 'data' as a 'json' expression: ????

url = 'https://httpbin.org'
dat = {None:[None,None], "0":{'':{'1':2,'2':1}, '2':[None,12], '\0':[None,[None,None]]}, 0:'1', '\0':[[1,2,3],2,3,4]}

r = requests.post(url+'/post', json=dat)
print(r.text)

# More to explore about the json Request attribute

{
  "args": {}, 
  "data": "{\"null\": [null, null], \"0\": {\"\": {\"1\": 2, \"2\": 1}, \"2\": [null, 12], \"\\u0000\": [null, [null, null]]}, \"0\": \"1\", \"\\u0000\": [[1, 2, 3], 2, 3, 4]}", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "142", 
    "Content-Type": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e9a-0d432a0f4fd911e73ece615a"
  }, 
  "json": {
    "\u0000": [
      [
        1, 
        2, 
        3
      ], 
      2, 
      3, 
      4
    ], 
    "0": "1", 
    "null": [
      null, 
      null
    ]
  }, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/post"
}



In [166]:
# post() request with files:
url = 'https://httpbin.org/post'
files = {'rdme': ('./README.md', open('./README.md', 'rb'))}

r = requests.post(url,files=files)
print(r.text)

# Q. What is the use of the first element of tuple in 'files/rdme' field?

# strings as files:
files = {'greet': 'This is a string to greet you.\nHave a wonderful day.'}
r = requests.post(url,files=files)
print(r.text)


# These standards and methods seem distinctive only in notation.
# Quote:
"""
It is strongly recommended that you open files in binary mode. 
This is because Requests may attempt to provide the 
Content-Length header for you, and if it does this value will be 
set to the number of bytes in the file. Errors may occur if you 
open the file in text mode.
"""

{
  "args": {}, 
  "data": "", 
  "files": {
    "rdme": "# assignments"
  }, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "160", 
    "Content-Type": "multipart/form-data; boundary=32cc45e09042bc920729082e60af01e6", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e9b-74de595156a41b556cbd3ef5"
  }, 
  "json": null, 
  "origin": "14.97.164.46", 
  "url": "https://httpbin.org/post"
}

{
  "args": {}, 
  "data": "", 
  "files": {
    "greet": "This is a string to greet you.\nHave a wonderful day."
  }, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "194", 
    "Content-Type": "multipart/form-data; boundary=809dbb4cd7e28aebe8613b3aef4b7614", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.31.0", 
    "X-Amzn-Trace-Id": "Root=1-65dd7e9c-2f105031168beb5c6a743d14"
  }, 
  "jso

'\nIt is strongly recommended that you open files in binary mode. \nThis is because Requests may attempt to provide the \nContent-Length header for you, and if it does this value will be \nset to the number of bytes in the file. Errors may occur if you \nopen the file in text mode.\n'

In [167]:
# A Response has a headers field ; It is of the generic type: dict(strlist(str))?

url = 'https://httpbin.org'
r = requests.head(url + '/get')

print("Type:",type(r.headers)) 
print(r.headers)

# Not dict() type but similar. Why?
    # Because it conforms to HTTP Standard that requires Response headers
    # be case-insensitive. (i.e. 'A' == 'a')
    # Multiple headers may be combined on common "headers/?" sub-fields.
        # eg. "name": "John" U "name": "Jack" <-> "name": ["John", "Jack"]

# type(r.headers) conforms to RFC 7230 HTTP standard for the headers field.

Type: <class 'requests.structures.CaseInsensitiveDict'>
{'Date': 'Tue, 27 Feb 2024 06:18:06 GMT', 'Content-Type': 'application/json', 'Content-Length': '306', 'Connection': 'keep-alive', 'Server': 'gunicorn/19.9.0', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Credentials': 'true'}


In [168]:
# HTTP Cookies:


In [169]:
# Redirection and History:

# Quote:
"""
By default Requests will perform location 
redirection for all verbs except HEAD.
"""

# The 'history' attribute of a Response accounts 
# for re-directions. It is of list(requests.models.Response) type.

url = 'http://github.com'
r = requests.get(url,data='HTTP4LYF', params={'YES':1,'NO':2})

print("Final redirected URL:",r.url)
print("Status Code of Response:",r.status_code)
print("Redirection history:",r.history,"; Type:", type(r.history))

# Type of history list items:
print("Element Type:",type(r.history[0])) # Response

Final redirected URL: https://github.com/?YES=1&NO=2
Status Code of Response: 200
Redirection history: [<Response [301]>] ; Type: <class 'list'>
Element Type: <class 'requests.models.Response'>


In [170]:
# Request 'timeout' field:

# It is of type 'float' ;
# If a Response is not *initiated* before 'timeout' seconds,
# a Timeout Exception is raised. If initiated, it may exceed 
# timeout seconds before being completely identified.

try:
    requests.get('http://github.com',timeout=0.0001)
except Exception as e:
    print(e)

HTTPConnectionPool(host='github.com', port=80): Max retries exceeded with url: / (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x111333f50>, 'Connection to github.com timed out. (connect timeout=0.0001)'))


In [146]:
# Sessions: (Concise Notation)
    
    # Request and Response fields/attributes contain  
    # redundant and related state over multiple instances.
    
    # Hence, the complete state/representation of Responses and Requests 
    # may be decomposed into a generic part, common to all instances and 
    # referenced by the term Session, along with the specific dynamic part 
    # that varies from one instance to another.
    
    # A Session may be declared once, and referenced 
    # by multiple Requests/Responses over which it is invariant.

# Q. Is a Request/Response a part of a Session or vice-versa?
    
    # I would consider a Session as a part of a Request/Response.

    # However, 1 Session corresponds to many Requests whereas,
    # 1 Request can only correspond to 1 Session;
    
# It depends on and varies with how we interpret nothing. Do we 
# fill in the "missing" details, or do we only assume what is given?

In [194]:
# A Session:
s = requests.Session()
print(type(s))

# Quote: "A Session object implements all of the main Request methods" ;
url = ('https://httpbin.org' + 
       '/cookies' + 
       '/set' + 
       '/sessioncookie' + 
       '/123456789')

s.get(url)
r = s.get('https://httpbin.org/cookies')

print(r.text)

<class 'requests.sessions.Session'>
{
  "cookies": {
    "sessioncookie": "123456789"
  }
}



In [ ]:
# The real question: Does the implementation assert the assumptions?
    
    # Realistically, one might argue electromagnetic waves may be defined
    # "independently" of our theory of Responses and Requests, yet it may 
    # be that one is simply a translation of the other in another language.

    # If the implementation that is reality, truly is independent of 
    # such theories and models, then it fails that the implementation which
    # we argue, indeed asserts the assumptions that we make about it.

    # Otherwise, one may describe Electromagnetic Field Theory in terms of
    # Responses and Requests and similar terms as discussed above.

    # However, one is infinitely elegant and beautiful, described in the 
    # language of Mathematics, while the other is..., well..., words 
    # are nothing but the meaning/reality we assume in them, a mockery 
    # of reality, a mockery of Mathematics. 

    # Is Mathematics a mockery of itself? Sometimes it feels like an 
    # infinitely long proof by contradiction of itself unfolding in time.
    # Mathematicians crafting it so by the time we contradict it, time 
    # itself becomes undefined.

    # Mathematics argues existence, and in Mathematics exists all that 
    # do not contradict their own existence. If it exists, it is identified 
    # by Mathematics, otherwise it is None, which is also identified in 
    # Mathematics.